In [ ]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as parquet

from scipy.ndimage import median_filter


In [ ]:
mask = fits.open("/home/bekah/gPhoton2/gPhoton/cal_data/FUV_mask.fits")

In [ ]:
plt.imshow(mask[0].data,  interpolation='none')

In [ ]:
mask[0].data.shape

In [ ]:
fmask = fits.open("/home/bekah/gPhoton2/gPhoton/cal_data/FUV_mask.fits")
plt.imshow(fmask[0].data,  interpolation='none')
plt.colorbar()

In [ ]:
nmask = fits.open("/media/bekah/BekahA/hotspot/200_stack_fuv/correct_stacks/derivatives/dilated_mask_filtered10_downsize_f.fits")
plt.imshow(1 - nmask[0].data,  interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(fmask[0].data-(1-nmask[0].data) )
plt.colorbar()

#### ideas for hotspot mask:
1. median filtering 
2.  thresholding: 

    a) what the mission did: 
>locate hotspots, defined as areas that have 5*sigma or higher rate than the
;diffuse background.  Assume Poisson statistics to calculate the 1-sigma
;level, because the original data has the hotspots and the median filtered
;data has an artificially low noise level.  Remember this part needs to be
;in COUNTS to scale properly to noise.
   hotidx = where((img - flatimg)/(sqrt(mean(flatimg))) ge 5,nhot)"""
   
    b) global thresholding, adaptive thresholding, and Otsu's method

In [ ]:
e30025 = fits.open("/media/bekah/BekahA/hotspot/17316_f500_image.fits")
image = e30025[0].data


In [ ]:
filtered_image = median_filter(image, size=5)


In [ ]:
from compare_aspect.plots import centile_clip

plt.imshow(centile_clip(filtered_image))
plt.colorbar()

In [ ]:
x = (3250, 3500)
y = (3020,3260)
subset = filtered_image[3200:3400,2800:3000]

In [ ]:
np.median(subset)

In [ ]:
# Exclude zero values
non_zero_arr = filtered_image[filtered_image != 0]

# Calculate median and mean
median = np.median(non_zero_arr)
mean = np.mean(non_zero_arr)
std = np.std(non_zero_arr)

In [ ]:
median

In [ ]:
np.max(fmask)

In [ ]:
import matplotlib.image

mask = (image <= 20) | (image > 600)
matplotlib.image.imsave('mask.png',mask)

In [ ]:
mask_small = image <= 20
mask_big = image >= 500

plt.imshow(mask_small+mask_big, cmap='Greys', interpolation='none')
plt.colorbar()

In [ ]:
mask_big = filtered_image > 30
plt.imshow(mask_big)
plt.colorbar()

In [ ]:
mask = mask_big + mask_small

plt.imshow(mask)
plt.colorbar()

In [ ]:
mask.shape

In [ ]:
t = 6750
eclipse = 30025

def make_mask(eclipse, t):
    """ combine high and low value masks for x different frames of dose maps for a single eclipse
    to make a hotspot mask. bc hotspots should be in the same location-ish, the masks combined will 
    have a value of x in the hotspot locations. """
    mask = np.empty([6000, 6000])
    for i in range(3): 
        dose = fits.open(f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-b00-f0025-t0{t+(i*750)}-g_dose.fits.gz")
        image = dose[0].data
        filtered_image = median_filter(image, size=5)
        
        mask_small = filtered_image <= 0 
        mask_big = filtered_image > 3
        
        mask = mask_big + mask_small + mask

    return mask 

mask = make_mask(eclipse, t)

In [ ]:
plt.imshow(mask)
plt.colorbar()

In [ ]:
def combine_three_masks(eclipse1, eclipse2, eclipse3): 
    e1 = fits.open(f"/media/bekah/BekahA/backplane_widths/e{eclipse1}/e{eclipse1}-nd-b00-f3000-t00000-g_dose.fits.gz")
    image1 = e1[0].data
    filtered_image1 = median_filter(image1, size=5)
    
    e2 = fits.open(f"/media/bekah/BekahA/backplane_widths/e{eclipse2}/e{eclipse2}-nd-b00-f3000-t00000-g_dose.fits.gz")
    image2 = e2[0].data
    filtered_image2 = median_filter(image2, size=5)
    
    
    e3 = fits.open(f"/media/bekah/BekahA/backplane_widths/e{eclipse3}/e{eclipse3}-nd-b00-f3000-t00000-g_dose.fits.gz")
    image3 = e3[0].data
    filtered_image3 = median_filter(image3, size=5)
    
    
    mask = (filtered_image1 <= 0) | (filtered_image1 > 10)
    mask2 = (filtered_image2 <= 0) | (filtered_image2 > 10)
    mask3 = (filtered_image3 <= 0) | (filtered_image3 > 10)
    
    mask = mask.astype(int)
    mask2 = mask2.astype(int)
    mask3 = mask3.astype(int)


    return mask + mask2 + mask3

In [ ]:
fmask = combine_three_masks(23346, 23334, 23328)

In [ ]:
import matplotlib.image

matplotlib.image.imsave('fmask_g1.png', fmask>=3)